In [1]:
import re
import pandas as pd
import numpy as np
from collections import Counter 
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
## 재 토큰화  # vocab_szie 1756
tokenizer = Tokenizer(1756, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

# 리뷰길이 25으로 패딩 , max_len = 25 확인 
X_train = pad_sequences(X_train, maxlen=25)

In [20]:
X_train02 =  pd.DataFrame(X_train)

with open('X_train02.csv', 'w', encoding = 'utf-8-sig') as f:
    X_train02.to_csv(f)
pd.DataFrame(X_train02)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,38,42,10,19
1,0,0,0,0,0,0,0,0,0,0,...,209,261,1,66,6,57,48,78,958,959
2,0,0,0,0,0,0,0,0,0,0,...,0,0,245,8,3,7,14,960,94,2
3,0,0,0,0,0,0,0,0,0,0,...,804,179,39,91,423,156,1,107,16,40
4,0,0,0,0,0,0,0,0,0,0,...,0,692,180,2,6,1,805,25,37,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3727,0,0,0,0,0,0,0,0,0,0,...,0,0,4,3,7,14,8,22,9,2
3728,0,0,0,0,0,0,0,0,0,0,...,193,4,68,46,44,47,36,26,9,2
3729,0,0,0,0,0,0,0,0,0,0,...,162,441,33,16,356,92,1,5,131,1
3730,0,0,0,0,0,0,0,0,0,0,...,0,41,36,4,40,200,1,107,16,383


In [26]:
X_train = pd.read_csv('X_train02.csv')
X_train.shape

(3732, 26)

In [24]:
# 기존 X_train csv 파일 전환 후 로드시 X_train.csv 파일로 틀어짐 전처리 
X_train = pd.read_csv('X_train02.csv')
X_train = X_train.drop('Unnamed: 0' , axis=1)
X_train2 = X_train["tokenized"].squeeze()
X_train3 = X_train2.values
X_train4 = []
for str in X_train3:
    str = str.replace("[", "").replace("]", "").replace(" '", "").replace("'", "")
    X_train4.append(str.split(","))
X_train = X_train4.copy()

KeyError: 'tokenized'

In [3]:
# 모델로드
loaded_model = load_model('test01_model.h5')

In [4]:
# 형태소 처리 라이브러리 정의 
okt =  Okt()\
# 토큰 라이브러리
tokenizer = Tokenizer()
# 불용어 : 추가시 리스트에 추가
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
## 리뷰예측하기
def sentiment_predict(new_sentence):
  new_sentence = new_sentence.replace("↑", "상승").replace("↓", "하락")
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = 25)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [5]:
sentiment_predict('루나코인 -99%↓ 투자자들 패닉')

61.52% 확률로 긍정 리뷰입니다.


In [6]:
sentiment_predict('유럽증시 전쟁연기 소식 15% ↑')

61.52% 확률로 긍정 리뷰입니다.


In [7]:
sentiment_predict('2호 회사 영업실적 최대 기대감 솔솔')

61.52% 확률로 긍정 리뷰입니다.


In [8]:
sentiment_predict('2호 회사 신규 상품 출시')

61.52% 확률로 긍정 리뷰입니다.
